# 문장 생성(koGPT2 모델이용)

### prompt build : prompt에 출력에 대한 설정을 포함시킨다

In [ ]:
# 0) 설치 (Colab)
!pip -q install transformers==4.42.3 torch --upgrade


In [ ]:
# 1) 모델/토크나이저 로딩 (★ KoGPT2는 fast 사용)
import torch, re, random
from typing import List
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_NAME = "skt/kogpt2-base-v2"

# ★ 핵심: use_fast=True (기본값, 명시해도 됨)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# GPT2계열은 pad_token이 없으므로 eos로 대체
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

DEVICE = 0 if torch.cuda.is_available() else -1
f
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=DEVICE
)


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
# 2) 프롬프트 템플릿 (톤/길이 옵션)
LENGTH_TO_TOKENS = {"짧게": 25, "중간": 45, "길게": 70}

TONE_SYSTEM = {
    "친근": "따뜻하고 친근한 말투로 존댓말을 사용하고, 쉬운 표현을 쓰며, 자연스럽게 말해주세요.",
    "격식": "격식 있고 전문적인 문체로, 정확하고 간결하게 서술하며, 불필요한 감탄이나 과장을 피하세요.",
    "마케팅": "마케팅 카피처럼 생동감 있게, 장점을 강조하고 행동을 유도하는 문구를 포함해 설득력 있게 작성하세요."
}

def build_prompt(keywords: List[str], tone: str, length_label: str) -> str:
    kw_str = ", ".join(keywords)
    tone_guide = TONE_SYSTEM.get(tone, TONE_SYSTEM["친근"])
    length_guide = f"문장 길이는 '{length_label}' 수준으로 조절하세요."
    prompt = (
        f"다음 지침을 따르세요.\n"
        f"1) 아래 키워드를 모두 포함해 한 문장만 작성합니다.\n"
        f"2) {tone_guide}\n"
        f"3) {length_guide}\n"
        f"4) 문장은 자연스러운 한국어로 마침표로 끝냅니다.\n\n"
        f"키워드: {kw_str}\n"
        f"문장:"
    )
    return prompt


In [ ]:
# 3) 생성 함수 (재시도 포함)
def _first_sentence_only(text: str) -> str:
    text = text.strip()
    for end_mark in ["다.", "요.", ".", "!", "?"]:
        pos = text.find(end_mark)
        if pos != -1:
            return text[:pos+len(end_mark)]
    return text

def _contains_all_keywords(sentence: str, keywords: List[str]) -> bool:
    s = sentence.lower()
    return all(kw.lower() in s for kw in keywords)

def generate_sentence(
    keywords: List[str],
    tone: str = "친근",               # "친근" | "격식" | "마케팅"
    length_label: str = "중간",       # "짧게" | "중간" | "길게"
    temperature: float = 0.9,
    top_p: float = 0.9,
    repetition_penalty: float = 1.1,
    no_repeat_ngram_size: int = 3,
    num_return_sequences: int = 5,
    max_tries: int = 4,
    seed: int = 42
):
    assert length_label in LENGTH_TO_TOKENS
    max_new_tokens = LENGTH_TO_TOKENS[length_label]
    prompt = build_prompt(keywords, tone, length_label)

    torch.manual_seed(seed)
    random.seed(seed)

    t, p = temperature, top_p
    last_outs = None

    for attempt in range(1, max_tries+1):
        outs = generator(
            prompt,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=t,
            top_p=p,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            num_return_sequences=num_return_sequences
        )
        last_outs = outs

        for o in outs:
            gen = o["generated_text"][len(prompt):].strip()
            sent = _first_sentence_only(gen)
            sent = re.sub(r"\s+", " ", sent).strip()
            if _contains_all_keywords(sent, keywords):
                return sent, {"attempt": attempt, "temperature": t, "top_p": p}

        # 재시도 시 탐색성 ↑
        t = min(1.2, t + 0.1)
        p = min(0.98, p + 0.05)

    # 모든 시도 실패 → 가장 자연스러운 후보 반환(경고 안내)
    fallback = _first_sentence_only(last_outs[0]["generated_text"][len(prompt):])
    fallback = re.sub(r"\s+", " ", fallback).strip()
    return f"[참고] 일부 키워드가 빠졌을 수 있습니다: {fallback}", {"attempt": attempt, "temperature": t, "top_p": p}


In [ ]:
# 4) 빔서치(선택)
def generate_beam(
    keywords: List[str],
    tone: str = "격식",
    length_label: str = "중간",
    num_beams: int = 5,
    repetition_penalty: float = 1.1,
    no_repeat_ngram_size: int = 3
):
    max_new_tokens = LENGTH_TO_TOKENS[length_label]
    prompt = build_prompt(keywords, tone, length_label)
    outs = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=num_beams,
        early_stopping=True,
        repetition_penalty=repetition_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        num_return_sequences=1
    )
    gen = outs[0]["generated_text"][len(prompt):].strip()
    sent = _first_sentence_only(gen)
    sent = re.sub(r"\s+", " ", sent).strip()
    return sent


In [ ]:
# 5) 사용 예시
examples = [
    (["소나기", "여름", "기억"], "친근", "짧게"),
    (["데이터", "모델", "성능"], "격식", "중간"),
    (["프리미엄", "혜택", "지금"], "마케팅", "길게")
]

for kws, tone, length_label in examples:
    sent, meta = generate_sentence(
        kws, tone=tone, length_label=length_label,
        temperature=0.9, top_p=0.9,
        num_return_sequences=6, max_tries=4, seed=42
    )
    print(f"\n[톤={tone} | 길이={length_label} | 키워드={kws}]")
    print("문장:", sent)
    print("메타:", meta)

print("\n[빔서치 예시]")
print(generate_beam(["연구", "결과", "의의"], tone="격식", length_label="중간"))



[톤=친근 | 길이=짧게 | 키워드=['소나기', '여름', '기억']]
문장: [참고] 일부 키워드가 빠졌을 수 있습니다: 화재, 폭발 위험 사용 빈도 - 최대 6명 이상 키워드: 소음 및 초더기 : 화염,
메타: {'attempt': 4, 'temperature': 1.2, 'top_p': 0.98}

[톤=격식 | 길이=중간 | 키워드=['데이터', '모델', '성능']]
문장: [참고] 일부 키워드가 빠졌을 수 있습니다: 데이터 등 소프트웨어를 많이 쓰는 사람, 특히 자료 수집의 중요성을 인지하는 사람들을 위한 단어입니다.
메타: {'attempt': 4, 'temperature': 1.2, 'top_p': 0.98}


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



[톤=마케팅 | 길이=길게 | 키워드=['프리미엄', '혜택', '지금']]
문장: 프리미, 혜택 2. 프리미엄(혜택), 현재, 지금, 현재. 한 단어씩 꼭 써야 한다.
메타: {'attempt': 2, 'temperature': 1.0, 'top_p': 0.9500000000000001}

[빔서치 예시]
연구, 결과 의의 내용: 연구결과 의의 자료: 연구결과, 의의 출처: 한국문화콘텐츠진흥원 (www.
